With this code I re-create images produced by MagneticFragmantentation and overplot the centroids of the fragments, per magnetogram.

I need to reproduce the same image I made with Magnetic Fragmentation, with the same threshold. I then need to restore the properties I saved out and get the info about the position of the centroids of the bounding boxes for each fragment. In the end, I want to plot the image and overplot the info of the bounding boxes.

Sidenote: Although data.date gives the 'date_obs' of the hmi, the hmi .fits file itself has been named according to 't_rec'.
't_rec' has a format of '2014.03.23_00:00:00_TAI'. If I want to convert this to datetime object I keep only the 20 first digits and convert: #date_meta=dateparser.parse(data.meta.get('t_rec')[0:19])

Import required modules and set up matplotlib display properties. Set up variables.

In [ ]:
# astropy.units allows us to use physical units throughout the code and easily
# perform calculations on them
import astropy.units as u
# sunpy.map handles the coordinates of the data we are using
import sunpy.map
from sunpy.coordinates import frames
# The SkyCoord object lets us convert between astronomical coordinate systems
from astropy.coordinates import SkyCoord
# matplotlib.pyplot is the standard plotting tool in python
import matplotlib.pyplot as plt
#importing csv module so I can read the output of write_props
import csv
from datetime import datetime
#mac seems to need glod to get rid of the .DS store file
import glob
#Readsav reads IDL .dat files
from scipy.io import readsav
#Dateparser needs installation: http://dateparser.readthedocs.io/en/latest/index.html
#It is very versatile but slows things down a lot
import dateparser
from datetime import timedelta
#Fraser's functions
from rotate_long import rotate_long
from window_corner_rotation import window_corner_rotation
# will use it to define a path and check if it exists
from pathlib import Path
    
# The next three lines are for plotting in a Jupyter notebook
# explanation of difference between: %matplotlib notebook and %matplotlib inline
# https://github.com/matplotlib/matplotlib/issues/4879
%matplotlib inline
plt.rcParams['figure.figsize'] = 11, 11
plt.rcParams.update({'font.size': 12})

# Choose threshold. Magnetic fields under this level (Gauss) will be ignored
threshold = 200 # Gauss

act_region='AR12010'
datapath = '/Users/dina/phd-mac/large_folders/Lucie_hmi_data/'
main_path = '/Users/dina/phd-mac/large_folders/MagneticFragmentation_github/output/'
#whole region
# bottom_left =[-120, -50] 
# top_right =[120, -230]
# images_path=main_path + 'bbox_centroid_flares_' + str(threshold) + 'G/'
# propspath= main_path + act_region + '_' + str(threshold) + 'G_fragment_properties/'
fl_props_path_dat='/Users/dina/Dropbox/PhD/phd/work/year3/codes_yr3/mag_flux_project/mag_flux_flarelists/fl_ar12010_properties.dat'

#subregion
#subregion
bottom_left_sub =[-50, -170] 
top_right_sub =[50, -100]
images_path= main_path
propspath= main_path + act_region + '_sub_' + str(threshold) + 'G_fragment_properties/'

First I need a sunpy object. Load the path with the hmi .fits files.

In [ ]:
files_to_load=glob.glob(datapath+"/*")
print(len(files_to_load))

Load the path with the properties of fragments .txt files

In [ ]:
props_to_load=glob.glob(propspath+"/*")
print(len(props_to_load))

Load the structure that contains information about all the RHESSI flares that happenned around the time the for loop will run later. 

In [ ]:
#Reading in the flarelist for that day, from a .dat form
#This might help: https://docs.python.org/3.3/reference/lexical_analysis.html#string-literals
f=readsav(fl_props_path_dat)

#See which arrays I have saved inside f
#[i for i in f]

In [ ]:
f.fl_etim

I need to load in the very first magnetogram. Its data and time will be stored. I will use this to transform the bottom left and top right points of the window from arcseconds to degrees.

In [ ]:
filename =files_to_load[0]

data = sunpy.map.Map(filename).rotate()
initial_obs_time = data.date #it's a datetime object, gives the 'date_obs' of the hmi

#transform pixel to arcseconds
bl = SkyCoord(bottom_left_sub[0] * u.arcsec,
              bottom_left_sub[1] * u.arcsec, frame=frames.Helioprojective, obstime=initial_obs_time)  
tr = SkyCoord(top_right_sub[0] * u.arcsec,
              top_right_sub[1] * u.arcsec, frame=frames.Helioprojective, obstime=initial_obs_time) 
#transform arcseconds to degrees
bl_stonyhurst = bl.transform_to(frames.HeliographicStonyhurst)
tr_stonyhurst = tr.transform_to(frames.HeliographicStonyhurst)

In [ ]:
#For loop for all magnetograms
for image in range(len(props_to_load)):#
    # Create the full filepath of the FITS file
    filename =files_to_load[image] #if I use glob

    # Pull out the data into a rotated,in a n angle according to the hmi header, SunPy map
    data = sunpy.map.Map(filename).rotate()
    print(data.date)
    
    # Transform the initial window coordinates into lat/long coordinates at the time of this image
    # Calculate the timedifference in the form [days.fraction_of_day] from the original image in days
    # The brackets at total_seconds() indicate that this is a function attached to the variable
    timediff = (data.date - initial_obs_time).total_seconds()/86400
    [bl, tr] = window_corner_rotation(bl_stonyhurst, tr_stonyhurst, timediff, data.date) 

    # Create a submap using those coordinates
    submap_area = data.submap(bl, tr)
    
    #I want to load the correct fragment properties file, for the fragments with negative and positive flux.
    #To do this I use the 'date-obs' from the hmi header, which is given from the data.date
    #I convert this to the exact format the properties files have been stored.
    neg_properties=propspath+'{0}'.format(data.date.strftime('%Y%m%d_%H%M%S'))+'_n.txt'
    pos_properties=propspath+'{0}'.format(data.date.strftime('%Y%m%d_%H%M%S'))+'_p.txt'
    
    #I define new arrays to read in what I want to use later.
    #Here, negative latitude and longitude; row and column, in degrees
    nc_lat=[] #negative centroids_latitude
    nc_lng=[] #negative centroids_longitude
    pc_lat=[] #positive centroids_latitude
    pc_lng=[] #positive centroids_longitude

    #read the properties in, if they exist. Else move on with the loop.
    my_file = Path(neg_properties)
    if my_file.is_file():
        with open(neg_properties,'r') as neg:
            filereader = csv.reader(neg, delimiter=',')
            for column in filereader:
                nc_lat.append(float(column[1]))
                nc_lng.append(float(column[2]))
    else:
        continue
    
        with open(pos_properties,'r') as pos:
            filereader = csv.reader(pos, delimiter=',')
            for column in filereader:
                pc_lat.append(float(column[1]))
                pc_lng.append(float(column[2]))  
            
    #From the flares I loaded before, I want only the one that happened around the time of each hmi magnetogram.    
    fl_lat=[]
    fl_lng=[]
    for i in range(len(f.fl_stim)):
        #Look at the time period 
        #[stim-time between two consecutive magnetograms, etim+time between two consecutive magnetograms]
        if dateparser.parse(str(f.fl_stim[i])) >= data.date-timedelta(minutes=24)\
        and dateparser.parse(str(f.fl_etim[i])) <= data.date+timedelta(minutes=24):
            fl_lat.append(f.fl_lat[i])
            fl_lng.append(f.fl_lng[i])
        
            
    #Plot out a map and save it
    #http://docs.sunpy.org/en/stable/generated/gallery/skip_magnetogram_active_regions.html
    #Plot the submap
    ax=plt.subplot(projection=submap_area)
    submap_area.plot()
    ax.set_autoscale_on(False) 
    
    #Overplot the centroids
    c = SkyCoord(nc_lng * u.deg, nc_lat* u.deg, frame="heliographic_stonyhurst")
    ax.plot_coord(c, 'yx')

    c = SkyCoord(pc_lng * u.deg, pc_lat * u.deg, frame="heliographic_stonyhurst")
    ax.plot_coord(c, 'bx')
        
    #Overplot the flares
    c = SkyCoord(fl_lng * u.deg, fl_lat * u.deg, frame="heliographic_stonyhurst")
    ax.plot_coord(c, 'ro', markersize=14)

    imagefilename = images_path + 'HMI_'+str(threshold)+'_G_' + str(image).zfill(4)
    plt.savefig(imagefilename)
    plt.clf()
 #   plt.show()
  